In [ ]:
%matplotlib inline

## torch.autograd 를 사용한 자동미분

신경망 학습시에는 **역전파** 알고리즘 주로 사용
-> 매개변수는 손실함수의 변화도 (gradient)에 따라 조정됨
-> 변화도 계산을 위해 pytorch 에서는 torch.autograd라는 자동 미분 엔진 사용
-> 모든 계산 그래프에 대해 변화도 자동계산 지원

In [ ]:
import torch

x = torch.ones(5) # input tensor
y = torch.zeros(3) # expected output
w = torch.randn(5, 3, requires_grad=True) # 기울기
b = torch.randn(3, requires_grad=True) # 상수
z = torch.matmul(x, w)+b # 연산
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y) # 손실

## Tensor, Function 과 연산 그래프

해당 코드는 ** 연산 그래프 ** 정의

w, b 는 최적화 해야하는 매개변수, 손실함수 변화도 계산 할 수 있어야 함. 해당 텐서에 requires_grad 속성 설정



In [ ]:
print ("Gradient function for z = ", z.grad_fn)
print ("Gradient function for loss = ", loss.grad_fn)

## 변화도 계산하기

매개변수 내에서 가중치 최적화 - > 매개변수 손실함수의 도함수 계산
loss.backward() 호출 후 값 가져오기

In [ ]:
loss.backward() # 호출 한번씩 밖에 안됨.
print(w.grad)
print(b.grad)

## 변화도 추적 멈추기

순전파 연산만 필요한 경우 -> torch.no_grad 연산으로 둘러싸 연산 추적 멈추기

In [ ]:
z = torch .matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

In [ ]:
# detach 메소드 적용해 같은 결과 얻기

z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

## 연산 그래프에 대한 추가 정보

DAG = directed acyclic graph

autograd는 데이터의 실행된 모든 연산들의 기록을 function,

순전파단계에서 autograd는 두가지 작업 수행

*   요청된 연산 수행해 결과 텐서 내기
*   DAG에 연산의 변화도 기능 유지

역전파 단계는 DAG의 root 에서 .backward() 호출될떄 시작.  
역전파 단계에서 autograd는

*   각 grad_fn 에서 변화도 계산
*   각 텐서의  .grad 속성에 계산 결과를 쌓고
*   연쇄법칙으로 모든 텐서에 propagate



## Optional Reading  텐서 변화도, 야코비안 곱

출력함수가 임의의 텐서라면 pytorch는 실제 변화도 아닌 야코비안 곱을 계산!


In [ ]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)